In [1]:
!pip install duckdb pandas pyarrow numpy faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 36.4 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from faker import Faker
import random

fake = Faker()
Faker.seed(42)
np.random.seed(42)
random.seed(42)

def generate_ecommerce_data(n_orders=50000, n_customers=10000, n_products=500):
    """
    Generate realistic e-commerce data with proper relationships
    """

    # Generate customers first since orders reference them
    customers = []
    for i in range(n_customers):
        signup_date = fake.date_between(start_date='-3y', end_date='-1d')
        customers.append({
            'customer_id': i + 1,
            'name': fake.name(),
            'email': fake.email(),
            'signup_date': signup_date,
            'country': fake.country_code(),
            'city': fake.city(),
            'customer_segment': np.random.choice(['Premium', 'Standard', 'Basic'], p=[0.2, 0.5, 0.3])
        })

    customers_df = pd.DataFrame(customers)

    # Generate product catalog
    categories = ['Electronics', 'Clothing', 'Home & Garden', 'Books',
                  'Sports', 'Toys', 'Beauty', 'Automotive']

    products = []
    for i in range(n_products):
        category = random.choice(categories)

        # Different categories have different price ranges
        price_ranges = {
            'Electronics': (50, 2000), 'Clothing': (15, 200),
            'Home & Garden': (20, 500), 'Books': (10, 50),
            'Sports': (25, 800), 'Toys': (10, 150),
            'Beauty': (15, 200), 'Automotive': (30, 1000)
        }

        min_price, max_price = price_ranges[category]
        base_price = round(np.random.uniform(min_price, max_price), 2)

        products.append({
            'product_id': i + 1,
            'product_name': f"{category} Item {i+1}",
            'category': category,
            'base_price': base_price,
            'cost': round(base_price * 0.6, 2),
            'stock_quantity': np.random.randint(0, 1000),
            'rating': round(np.random.uniform(3.0, 5.0), 1)
        })

    products_df = pd.DataFrame(products)

    # Generate orders and order items
    orders = []
    order_items = []
    order_id = 1

    for _ in range(n_orders):
        # Pick a random customer
        customer_id = np.random.randint(1, n_customers + 1)
        customer = customers_df[customers_df['customer_id'] == customer_id].iloc[0]

        # Order date must be after customer signup
        order_date = fake.date_time_between(
            start_date=customer['signup_date'],
            end_date='now'
        )

        # Premium customers tend to order more items
        if customer['customer_segment'] == 'Premium':
            n_items = np.random.choice([1, 2, 3, 4, 5], p=[0.2, 0.3, 0.25, 0.15, 0.1])
        else:
            n_items = np.random.choice([1, 2, 3, 4, 5], p=[0.5, 0.3, 0.12, 0.05, 0.03])

        # Select random products for this order
        selected_products = np.random.choice(products_df['product_id'].values, size=n_items, replace=False)

        order_total = 0
        for product_id in selected_products:
            product = products_df[products_df['product_id'] == product_id].iloc[0]
            quantity = np.random.choice([1, 1, 1, 2, 2, 3], p=[0.5, 0.2, 0.1, 0.1, 0.05, 0.05])

            # Occasionally apply discounts
            price = product['base_price']
            if np.random.random() < 0.15:
                discount_pct = np.random.uniform(0.05, 0.25)
                price = round(price * (1 - discount_pct), 2)

            item_total = round(price * quantity, 2)
            order_total += item_total

            order_items.append({
                'order_id': order_id,
                'product_id': product_id,
                'quantity': quantity,
                'unit_price': price,
                'item_total': item_total
            })

        payment_status = np.random.choice(['completed', 'failed', 'pending'], p=[0.95, 0.03, 0.02])

        orders.append({
            'order_id': order_id,
            'customer_id': customer_id,
            'order_date': order_date,
            'order_total': round(order_total, 2),
            'shipping_cost': round(np.random.uniform(5, 20), 2),
            'payment_status': payment_status,
            'shipping_country': customer['country']
        })

        order_id += 1

    orders_df = pd.DataFrame(orders)
    order_items_df = pd.DataFrame(order_items)

    return customers_df, products_df, orders_df, order_items_df

# Generate the dataset
customers_df, products_df, orders_df, order_items_df = generate_ecommerce_data()

print(f"Generated {len(customers_df):,} customers")
print(f"Generated {len(products_df):,} products")
print(f"Generated {len(orders_df):,} orders")
print(f"Generated {len(order_items_df):,} order items")

Generated 10,000 customers
Generated 500 products
Generated 50,000 orders
Generated 98,956 order items


In [3]:
# Save tables as Parquet files
customers_df.to_parquet('customers.parquet', engine='pyarrow', compression='snappy')
products_df.to_parquet('products.parquet', engine='pyarrow', compression='snappy')
orders_df.to_parquet('orders.parquet', engine='pyarrow', compression='snappy')
order_items_df.to_parquet('order_items.parquet', engine='pyarrow', compression='snappy')

# Compare with CSV to see the difference
customers_df.to_csv('customers.csv', index=False)
orders_df.to_csv('orders.csv', index=False)

import os

def get_size_mb(filename):
    return os.path.getsize(filename) / (1024 * 1024)

print("Storage Comparison:")
print(f"customers.csv:     {get_size_mb('customers.csv'):.2f} MB")
print(f"customers.parquet: {get_size_mb('customers.parquet'):.2f} MB")
print(f"Savings: {(1 - get_size_mb('customers.parquet')/get_size_mb('customers.csv'))*100:.1f}%\n")

print(f"orders.csv:        {get_size_mb('orders.csv'):.2f} MB")
print(f"orders.parquet:    {get_size_mb('orders.parquet'):.2f} MB")
print(f"Savings: {(1 - get_size_mb('orders.parquet')/get_size_mb('orders.csv'))*100:.1f}%")

Storage Comparison:
customers.csv:     0.73 MB
customers.parquet: 0.38 MB
Savings: 48.5%

orders.csv:        3.01 MB
orders.parquet:    1.25 MB
Savings: 58.5%


In [4]:
import duckdb

# Create a DuckDB connection
con = duckdb.connect(database=':memory:')

# Query the Parquet file directly
query = """
SELECT
    customer_segment,
    COUNT(*) as num_customers,
    COUNT(*) * 100.0 / SUM(COUNT(*)) OVER () as percentage
FROM 'customers.parquet'
GROUP BY customer_segment
ORDER BY num_customers DESC
"""

result = con.execute(query).fetchdf()
print("Customer Distribution:")
print(result)

Customer Distribution:
  customer_segment  num_customers  percentage
0         Standard           5070       50.70
1            Basic           2887       28.87
2          Premium           2043       20.43


In [6]:
query = """
SELECT
    strftime(o.order_date, '%Y-%m') as month,
    c.customer_segment,
    COUNT(DISTINCT o.order_id) as num_orders,
    COUNT(DISTINCT o.customer_id) as unique_customers,
    ROUND(SUM(o.order_total), 2) as total_revenue,
    ROUND(AVG(o.order_total), 2) as avg_order_value
FROM 'orders.parquet' AS o
JOIN 'customers.parquet' AS c
  ON o.customer_id = c.customer_id
WHERE o.payment_status = 'completed'
GROUP BY month, c.customer_segment
ORDER BY month DESC, total_revenue DESC
LIMIT 15
"""

monthly_revenue = con.execute(query).fetchdf()
print("Recent Monthly Revenue by Segment:")
print(monthly_revenue.to_string(index=False))

Recent Monthly Revenue by Segment:
  month customer_segment  num_orders  unique_customers  total_revenue  avg_order_value
2026-01         Standard        2600              1468     1683223.68           647.39
2026-01            Basic        1585               857     1031126.44           650.55
2026-01          Premium         970               560      914105.61           942.38
2025-12         Standard        2254              1571     1533076.22           680.16
2025-12          Premium         885               613      921775.85          1041.55
2025-12            Basic        1297               876      889270.86           685.64
2025-11         Standard        1795              1359     1241006.08           691.37
2025-11          Premium         725               554      717625.75           989.83
2025-11            Basic        1012               767      682270.44           674.18
2025-10         Standard        1646              1296     1118400.61           679.47
2025-10 

In [8]:
query = """
SELECT
    p.category,
    c.customer_segment,
    COUNT(DISTINCT oi.order_id) as num_orders,
    SUM(oi.quantity) as units_sold,
    ROUND(SUM(oi.item_total), 2) as total_revenue,
    ROUND(AVG(oi.item_total), 2) as avg_item_value
FROM 'order_items.parquet' oi
JOIN 'orders.parquet' o ON oi.order_id = o.order_id
JOIN 'products.parquet' p ON oi.product_id = p.product_id
JOIN 'customers.parquet' c ON o.customer_id = c.customer_id -- Added join with customers table
WHERE o.payment_status = 'completed'
GROUP BY p.category, c.customer_segment
ORDER BY total_revenue DESC
LIMIT 20
"""

category_analysis = con.execute(query).fetchdf()
print("Revenue by Category and Customer Segment:")
print(category_analysis.to_string(index=False))

Revenue by Category and Customer Segment:
     category customer_segment  num_orders  units_sold  total_revenue  avg_item_value
  Electronics         Standard        4729      6431.0     6638814.75         1299.18
  Electronics          Premium        2597      3723.0     3816429.62         1292.39
  Electronics            Basic        2685      3566.0     3585652.92         1240.28
   Automotive         Standard        4506      5926.0     3050679.12          633.18
       Sports         Standard        5049      6898.0     2745487.54          497.55
   Automotive          Premium        2557      3636.0     1883927.99          655.96
   Automotive            Basic        2497      3371.0     1727850.21          641.85
       Sports          Premium        2768      3930.0     1559640.96          492.00
Home & Garden         Standard        4527      6055.0     1535650.63          314.55
       Sports            Basic        2796      3768.0     1533942.41          508.60
Home & Garde

In [9]:
import time

# Analytical task: Calculate customer purchase patterns
print("Performance Comparison: Customer Purchase Analysis\n")

# Method 1: Using pandas
start_time = time.time()

# Merge dataframes
merged = order_items_df.merge(orders_df, on='order_id')
merged = merged.merge(products_df, on='product_id')

# Filter completed orders
completed = merged[merged['payment_status'] == 'completed']

# Group and aggregate
customer_patterns = completed.groupby('customer_id').agg({
    'order_id': 'nunique',
    'product_id': 'nunique',
    'item_total': ['sum', 'mean'],
    'category': lambda x: x.mode()[0] if len(x) > 0 else None
})

customer_patterns.columns = ['num_orders', 'unique_products', 'total_spent', 'avg_spent', 'favorite_category']
customer_patterns = customer_patterns.sort_values('total_spent', ascending=False).head(100)

pandas_time = time.time() - start_time

# Method 2: Using DuckDB
start_time = time.time()

query = """
SELECT
    o.customer_id,
    COUNT(DISTINCT oi.order_id) as num_orders,
    COUNT(DISTINCT oi.product_id) as unique_products,
    ROUND(SUM(oi.item_total), 2) as total_spent,
    ROUND(AVG(oi.item_total), 2) as avg_spent,
    MODE(p.category) as favorite_category
FROM 'order_items.parquet' oi
JOIN 'orders.parquet' o ON oi.order_id = o.order_id
JOIN 'products.parquet' p ON oi.product_id = p.product_id
WHERE o.payment_status = 'completed'
GROUP BY o.customer_id
ORDER BY total_spent DESC
LIMIT 100
"""

duckdb_result = con.execute(query).fetchdf()
duckdb_time = time.time() - start_time

print(f"Pandas execution time:  {pandas_time:.4f} seconds")
print(f"DuckDB execution time:  {duckdb_time:.4f} seconds")
print(f"Speedup: {pandas_time/duckdb_time:.1f}x faster with DuckDB\n")

print("Top 5 customers by total spent:")
print(duckdb_result.head().to_string(index=False))

Performance Comparison: Customer Purchase Analysis

Pandas execution time:  1.9872 seconds
DuckDB execution time:  0.1171 seconds
Speedup: 17.0x faster with DuckDB

Top 5 customers by total spent:
 customer_id  num_orders  unique_products  total_spent  avg_spent favorite_category
        8747           8               24     21103.21     879.30       Electronics
         617           9               27     19596.22     725.79       Electronics
        2579           9               18     17011.30     895.33            Sports
        6242           7               23     16781.11     729.61       Electronics
        5443           8               22     16697.02     758.96        Automotive


In [10]:
def analyze_product_performance(con, category=None, min_revenue=None, date_from=None, top_n=20):
    """
    Analyze product performance with flexible filtering.

    This demonstrates how to build reusable analytical queries that can be
    parameterized for different use cases. In production, you'd build a library
    of these functions for common analytical questions.
    """

    # Build the WHERE clause dynamically based on parameters
    where_clauses = ["o.payment_status = 'completed'"]

    if category:
        where_clauses.append(f"p.category = '{category}'")

    if date_from:
        where_clauses.append(f"o.order_date >= '{date_from}'")

    where_clause = " AND ".join(where_clauses)

    # Main analytical query
    query = f"""
    WITH product_metrics AS (
        SELECT
            p.product_id,
            p.product_name,
            p.category,
            p.base_price,
            COUNT(DISTINCT oi.order_id) as times_ordered,
            SUM(oi.quantity) as units_sold,
            ROUND(SUM(oi.item_total), 2) as total_revenue,
            ROUND(AVG(oi.unit_price), 2) as avg_selling_price,
            ROUND(SUM(oi.item_total) - (p.cost * SUM(oi.quantity)), 2) as profit
        FROM 'order_items.parquet' oi
        JOIN 'orders.parquet' o ON oi.order_id = o.order_id
        JOIN 'products.parquet' p ON oi.product_id = p.product_id
        WHERE {where_clause}
        GROUP BY p.product_id, p.product_name, p.category, p.base_price, p.cost
    )
    SELECT
        *,
        ROUND(100.0 * profit / total_revenue, 2) as profit_margin_pct,
        ROUND(avg_selling_price / base_price, 2) as price_realization
    FROM product_metrics
    """

    # Add revenue filter if specified
    if min_revenue:
        query += f" WHERE total_revenue >= {min_revenue}"

    query += f"""
    ORDER BY total_revenue DESC
    LIMIT {top_n}
    """

    return con.execute(query).fetchdf()

# Example 1: Top electronics products
electronics = analyze_product_performance(con, category='Electronics', top_n=10)
print("Top 10 Electronics Products:")
print(electronics[['product_name', 'units_sold', 'total_revenue', 'profit_margin_pct']].to_string(index=False))

# Example 2: High-revenue products across all categories
print("\n\nHigh-Revenue Products (>$50k revenue):")
high_revenue = analyze_product_performance(con, min_revenue=50000, top_n=10)
print(high_revenue[['product_name', 'category', 'total_revenue', 'profit']].to_string(index=False))

Top 10 Electronics Products:
        product_name  units_sold  total_revenue  profit_margin_pct
Electronics Item 113       262.0      510331.81              38.57
Electronics Item 154       289.0      486307.74              38.28
Electronics Item 122       229.0      448680.64              38.88
Electronics Item 472       251.0      444680.20              38.51
Electronics Item 368       222.0      424057.14              38.96
Electronics Item 241       219.0      407648.10              38.75
Electronics Item 410       243.0      400078.65              38.31
Electronics Item 104       233.0      400036.84              38.73
  Electronics Item 2       213.0      382583.85              38.76
Electronics Item 341       240.0      376722.94              38.94


High-Revenue Products (>$50k revenue):
        product_name    category  total_revenue    profit
Electronics Item 113 Electronics      510331.81 196846.19
Electronics Item 154 Electronics      486307.74 186140.78
Electronics Item 12